In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import networkx as nx

from collections import defaultdict

import scipy.sparse as spsparse
import matplotlib.pylab as plt


import pyscisci.all as pyscisci



In [2]:
path2aps = '/home/ajgates/APS'
path2aps = '/Volumes/GatesExpansionDrive/DataSets/APS/APS2019'

myaps = pyscisci.APS(path2aps, keep_in_memory=False)

# NOTE: APS does not contain disambiguated author or affiliation information by default, although researchers 
# have produced their own disambiguation to supplement the raw data

# Here, we include the author disambiguation used in Sinatra et al. (2016)
# if you didn't already download the file, uncomment the line below
#myaps.download_from_source(files_to_download='paa_supplement')
myaps.set_new_data_path(dataframe_name='paa ', new_path='publicationauthoraffiliation_supp2010')


In [3]:
pub  = myaps.pub 

# limit the publications to those published on/before 1966 
pub  = pub .loc[pub ['Year'] <= 1966]

# get their references
pub2ref  = myaps.load_references(filter_dict={'CitingPublicationId':np.sort(pub ['PublicationId'].unique())})

# and get their authors
pub2author  = myaps.load_publicationauthoraffiliation(columns = ['PublicationId', 'AuthorId', 'FullName'],
    filter_dict={'PublicationId':np.sort(pub ['PublicationId'].unique())})
  
aid2name = {aid:name for aid, name in pub2author [['AuthorId', 'FullName']].values}
del pub2author ['FullName']

In [4]:
sc, author2int = pyscisci.diffusion_of_scientific_credit(pub2ref , pub2author , 
                                        pub =pub , alpha = 0.9, max_iter = 100, tol = 1.0e-10)

int2aid = {i:aid for aid, i in author2int.items()}

print(sc.shape)

(22015,)


In [5]:
# print the top k authors
# Note: here we use an algorithmicly disambiguated author careers.  The original paper just
# disambiguated authors based on unique name.  So we expect the rankings to differ.

topk = 10

topk_authors = np.argpartition(sc, -topk)[-topk:]
topk_authors = topk_authors[np.argsort(sc[topk_authors])][::-1]

for int_id in topk_authors:
    print(aid2name[int2aid[int_id]], sc[int_id])


hans.a..bethe 0.008774540389162876
john.c..slater 0.008551603901008543
g...breit 0.007495337492029187
j..s..schwinger 0.006353349118286071
eugene.p..wigner 0.005233086936687401
robert.a..millikan 0.005045043029409964
robert.s..mulliken 0.004200793335143117
arthur.h..compton 0.004038834819293501
irving..langmuir 0.004025411535935083
john.h..van vleck 0.004013967086421246
